In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

TARGET_VARIABLES = ['appearance', 'aroma', 'overall', 'palate',
       'taste']
def pad_ones_column(X):
    """Add columns of ones to a data matrix"""
    return np.c_[ np.ones(len(ORIGINAL_DF)), X ]

def estimate_beta(X, y):
    """Compte estimates for beta given X and y
    
    Beta = (X^T X)^-1 X^T y
    """
    t1 = np.linalg.inv(np.matmul(X.T, X))
    t2 = np.matmul(X.T, y)
    return np.matmul(t1, t2)

def run_complete_linear_regression(X, y, beta_hat):
    """Run linear regression and produce error statistics"""
    # Make predictions based on testing data
    predicted_y = np.matmul(X, beta_hat)
    
    # Compute the error from the actual data
    errors = np.square(y - predicted_y)
    
    # Compute mean squared error
    MSE = np.mean(errors)
    
    # Compute sum squared error
    SSE = np.sum(errors)

    return MSE, SSE

def forward_stepwise_selection(df, target, all_variables, to_minimize, max_vars, eps, **kwargs):
    available_vars = set(all_variables)
    min_vars = len(available_vars) - max_vars
    current_vars = set()
    targets = np.matrix(df[[target]])
    #print('Min_vars',min_vars)
    
    saved_results = list()
    last_result = None
    
    def get_result(current_vars, new_var):
        temp_vars = current_vars | { new_var }
            
        data_matrix = pad_ones_column(np.matrix(df[list(temp_vars)]))
        (np.matrix(df[list(temp_vars)]))
        #print('new_var:', new_var)
        #print('temp_vars:',temp_vars)
        try:
            result = to_minimize(data_matrix, targets, **kwargs)
        except:
            return np.inf
        
        #print('Result:',result)
        return result
    
    while len(available_vars) > min_vars:
        temp_vars = list(available_vars)
        #print('Current vars:', current_vars)
        #print('Temp_vars:', temp_vars)
            
        results = np.fromiter(map(lambda var: get_result(current_vars, var), temp_vars),
                             float)
        
        if last_result is None:
            best_result = np.min(results)
            best_var = temp_vars[np.argmin(results)]
            
            last_result = best_result
            saved_results.append((best_var, best_result))
            
            current_vars = { best_var } | current_vars
            available_vars = available_vars - { best_var }
        else:
            best_results = results[(last_result - results)  > eps]
            
            if len(best_results) > 0:
                best_result = np.min(results)
                best_var = temp_vars[np.argmin(results)]
            
                last_result = best_result
                saved_results.append((best_var, best_result))
                
                current_vars = { best_var } | current_vars
                available_vars = available_vars - { best_var }
            else:
                break
            
    return saved_results

def bayesian_information_criterion(X, y, s_squared_est=True):
    # Number of variables used, subtracting the padded ones column
    d = X.shape[1] - 1
    N = X.shape[0]
    
    beta_hat = estimate_beta(X, y)
    predicted_y = np.matmul(X, beta_hat)
    MSE, SSE = run_complete_linear_regression(X, y, beta_hat)

    if s_squared_est:
        sigma_squared_hat = np.sum(np.square(predicted_y - np.mean(predicted_y))) / (N - 1)
    else:
        sigma_squared_hat = MSE

    t1 = np.power(np.log(2 * np.pi * sigma_squared_hat), N / 2)
    t2 = (SSE / (2 * sigma_squared_hat))
    t3 = ((d + 2) / N) * np.log(N)
    
#     bic = t1 + t2 + t3
    bic = t2 + t3
    
    return bic

In [2]:
def run_k_fold(X, y, n_splits=20):
    """Run k-fold cross validation"""
    kf = KFold(n_splits=n_splits)
    storeResultsMSE = []
    storeResultsSSE = []

    for i, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        beta_hat = estimate_beta(X_train, y_train)

        test_MSE, test_SSE = run_complete_linear_regression(X_test, y_test, beta_hat)

        storeResultsMSE.append(test_MSE)
        storeResultsSSE.append(test_SSE)

        #print("Iteration:", i, "MSE: ", test_MSE, "SSE:", test_SSE)  
    print("Average MSE:", np.mean(storeResultsMSE), "Average SSE:", np.mean(storeResultsSSE) )
    return np.mean(storeResultsMSE)
    
def run_predict_complete_prediction(df, target, variables):
    """Run the complete prediction workflow,
    targeting the given variable using only the variables specified."""
    
    assert len(variables) > 0
    
    targets = np.matrix(df[[target]])
    data_matrix = np.matrix(df[variables])
    
#     data_matrix = center_scale(data_matrix)
    data_matrix = pad_ones_column(data_matrix)
    
#     targets = center_scale(targets)
    
    avg_mse = run_k_fold(data_matrix, targets)
    
    return avg_mse

In [26]:
ORIGINAL_DF = pd.read_csv('train.csv')
INDEPENDENT_VARIABLES = [col for col in ORIGINAL_DF.columns if col not in TARGET_VARIABLES]

In [27]:
for t in TARGET_VARIABLES:
    res = forward_stepwise_selection(ORIGINAL_DF, t, INDEPENDENT_VARIABLES,
                           bayesian_information_criterion, 179,50 )
    bic = [r[1] for r in res]
    x_vals = [y for y in range(0,len(res))]
    x_labs = [r[0] for r in res]

    fig = plt.figure()
    plt.plot(x_vals,bic)
    plt.ylabel('BIC')
    plt.xticks(x_vals,x_labs,rotation = 90)
    plt.title('BIC vs Features: '+ t)
    plt.tight_layout()
    fig.savefig('BIC_REG_' + t + '.jpg')
    plt.close()


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:109: RuntimeWarning: overflow encountered in power


In [5]:
ORIGINAL_DF = pd.read_csv('train_not_reg.csv')
INDEPENDENT_VARIABLES = [col for col in ORIGINAL_DF.columns if col not in TARGET_VARIABLES]

In [9]:

selected_vars = []
for t in TARGET_VARIABLES:
    res = forward_stepwise_selection(ORIGINAL_DF, t, INDEPENDENT_VARIABLES,
                           bayesian_information_criterion, 179,50 )
    bic = [r[1] for r in res]
    x_vals = [y for y in range(0,len(res))]
    x_labs = [r[0] for r in res]
    selected_vars.append(x_labs)
    fig = plt.figure()
    print(t)
    plt.plot(x_vals,bic)
    plt.ylabel('BIC')
    plt.xticks(x_vals,x_labs,rotation = 90)
    plt.title('BIC vs Features: '+ t)
    plt.tight_layout()
    fig.savefig('BIC_NOT_REG_' + t + '.jpg')
    plt.close()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:111: RuntimeWarning: overflow encountered in power


appearance
aroma
overall
palate
taste


In [10]:
# Best Model (Reg)
# TARGET_VARIABLES = ['appearance', 'aroma', 'overall', 'palate', 'taste']
# Appearance
run_predict_complete_prediction(ORIGINAL_DF,'appearance',selected_vars[0])

# Aroma
run_predict_complete_prediction(ORIGINAL_DF,'aroma',selected_vars[1])

# Overall
run_predict_complete_prediction(ORIGINAL_DF,'overall',selected_vars[2])

# Palate
run_predict_complete_prediction(ORIGINAL_DF,'palate',selected_vars[3])
dfs
# Taste
run_predict_complete_prediction(ORIGINAL_DF,'taste',selected_vars[4])

Average MSE: 0.224659704117 Average SSE: 421.236945219
Average MSE: 0.249082230926 Average SSE: 467.029182986
Average MSE: 0.311401974429 Average SSE: 583.878702055
Average MSE: 0.251150661925 Average SSE: 470.907491109
Average MSE: 0.261093929353 Average SSE: 489.551117537


0.26109392935308068

In [19]:
(", ").join(selected_vars[4])


'userBias, avg_sent, ABV, #pos_words, Rauchbier, American Double / Imperial Stout, exclamations, #pos/#neg, #neg_words, American Malt Liquor, Netherlands, Euro Strong Lager, Euro Pale Lager, char_count, Poland, Munich Helles Lager, American Pale Ale (APA)'